In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time
import datetime

In [ ]:
def notinlst(lstall,lst):
    empty=[]
    for i in lstall:
        if i not in lst:
            empty.append(i)
    return empty

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
chargelst=[]
addrlst=[]
otherlst=[]
slowcountlst=[]

# 카카오맵 지도 접속하기
driver.get("https://map.kakao.com/")


html=driver.page_source
soup=BeautifulSoup(html,'html.parser')

# 검색창에 "검색어" 입력하기
searchbox = driver.find_element(By.CSS_SELECTOR,("#search\.keyword\.query"))
searchbox.send_keys("용인시 전기차 충전소")

# 검색버튼 누르기
searchbutton = driver.find_element(By.CSS_SELECTOR,"#search\.keyword\.submit")
searchbutton.send_keys(Keys.ENTER)



time.sleep(1)
a=driver.find_elements(By.CLASS_NAME,"link_name")
b=driver.find_elements(By.CLASS_NAME,"addr")
for i in range(len(a)):
    park=a[i].text
    addr=b[i].text
    addr=addr.splitlines()[0]
    #print(park,addr)
    chargelst.append(park)
    addrlst.append(addr)

#other information
c=driver.find_elements(By.CLASS_NAME,"moreview")
hreflst=[]
for item in c:
    href = item.get_attribute('href')
    hreflst.append(href)

time.sleep(1)

#더보기버튼 누르기(한번만 시행하면 됩니다)
morebutton = driver.find_element(By.CSS_SELECTOR,"#info\.search\.place\.more")
morebutton.send_keys(Keys.ENTER)

#모든 페이지에 대해 반복
for page in range(2,35):
    print(page)
    if page%5==1:
        rightbutton = driver.find_element(By.XPATH, "/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/button[2]")
        rightbutton.send_keys(Keys.ENTER)
        time.sleep(0.3)
        pageidx=1
        print("next")
    elif page%5 == 0:
        pageidx=5
    else:
        pageidx=page%5
    css="#info\.search\.page\.no{}".format(pageidx)
    time.sleep(0.5)
    nextbutton=driver.find_element(By.CSS_SELECTOR,css)
    nextbutton.send_keys(Keys.ENTER)
    time.sleep(0.3)
    a=driver.find_elements(By.CLASS_NAME,"link_name")
    b=driver.find_elements(By.CLASS_NAME,"addr")
    #print(a[0].text,b[0].text)
    time.sleep(0.3)
    for i in range(len(a)):
        park=a[i].text
        addr=b[i].text
        addr=addr.splitlines()[0]
        print(park,addr)
        chargelst.append(park)
        addrlst.append(addr)

    c = driver.find_elements(By.CLASS_NAME, "moreview")
    for item in c:
        href = item.get_attribute('href')
        hreflst.append(href)


#상세보기
for i in hreflst:
    slowcount=0
    time.sleep(1)
    driver.get(i)
    time.sleep(1)                    #너무 많이 호출할경우 카카오맵 자체에서 엑세스를 거부?
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    d=driver.find_elements(By.CLASS_NAME,"list_specific")
    if len(d)==0:
        print("no")
        otherlst.append('no')
        slowcountlst.append(slowcount)
        continue
    for idx in range(len(d)):
        print(d[idx].text)
        otherlst.append(d[idx].text)
    eall=driver.find_elements(By.CLASS_NAME,"set_item")
    e=driver.find_elements(By.CLASS_NAME,"set_item.item_disabled")
    ee=notinlst(eall,e)
    for i in range(len(ee)):
        try:
            if ee[1].text=="완속":
                slowcount+=1
        except:
            break
    for i in range(3,8):
        try:
            btn=driver.find_elements(By.CSS_SELECTOR,'#mArticle > div.cont_ecarcharge > div > div > div > a:nth-child({})'.format(i))
            btn[0].send_keys(Keys.ENTER)
            time.sleep(1)
            eall=driver.find_elements(By.CLASS_NAME,"set_item")
            e=driver.find_elements(By.CLASS_NAME,"set_item.item_disabled")
            ee=notinlst(eall,e)
            for i in range(len(ee)):
                if ee[i].text=="완속":
                    slowcount+=1
        except:
            slowcountlst.append(slowcount)
            print(slowcount)
            break

In [ ]:
print(len(chargelst))
print(len(addrlst))
print(len(otherlst))
print(len(slowcountlst))

In [ ]:
import pandas as pd
df5=pd.DataFrame(chargelst)
df5['addr']=addrlst
df5['time']=otherlst
df5['numslow']=slowcountlst
df5.to_csv("chargeat4pm.csv",encoding='euc-kr')

In [ ]:
dfhref=pd.DataFrame(hreflst)
dfhref.to_csv("href.csv",encoding='euc-kr')

In [ ]:
#iterate crawling every 2hours

In [ ]:
for t in range(1,41,2):
    hour=t
    if t>=24:
        hour=t-24
    while not(datetime.now().hour==hour ):
        print("wait 2hours...")
        time.sleep(300)
    atspecifictime=[]
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    for i in hreflst:
        slowcount=0
        time.sleep(1)
        driver.get(i)
        time.sleep(1)                    #너무 많이 호출할경우 카카오맵 자체에서 엑세스를 거부?
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        d=driver.find_elements(By.CLASS_NAME,"list_specific")
        if len(d)==0:
            print("no")
            atspecifictime.append('no')
            continue
        for idx in range(len(d)):
            print(d[idx].text)
            atspecifictime.append(d[idx].text)

    df5=pd.DataFrame(atspecifictime)
    df5.to_csv("chargeat{}pm.csv".format(t),encoding='euc-kr')

In [ ]:
#deal with exception

In [ ]:
def notchargable(example):
    errcount=0
    time.sleep(1)
    driver.get(example)
    time.sleep(1)                    #너무 많이 호출할경우 카카오맵 자체에서 엑세스를 거부?
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        for tr in range(1,6):
            e=driver.find_elements(By.XPATH,'//*[@id="mArticle"]/div[3]/div/div/table/tbody/tr[{}]/td[1]'.format(tr))
            print(e[0].text)
            if e[0].text[:4]=="충전불가":
                errcount+=1
    except:
        return errcount
    for j in range(3,8):
        try:
            time.sleep(1)
            btn=driver.find_elements(By.CSS_SELECTOR,'#mArticle > div.cont_ecarcharge > div > div > div > a:nth-child({})'.format(j))
            btn[0].send_keys(Keys.ENTER)
            time.sleep(1)
            for tr in range(1,6):
                try:
                    e=driver.find_elements(By.XPATH,'//*[@id="mArticle"]/div[3]/div/div/table/tbody/tr[{}]/td[1]'.format(tr))
                    print(e[0].text)
                    if e[0].text[:4]== "충전불가":
                        errcount+=1
                except:
                    break
        except:
            return errcount

In [ ]:
#위에것 실행결과에서 len(errcount)=48이 나옵니다

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
errcountlst=[]
for i in hreflst:
    result=notchargable(i)
    errcountlst.append(result)

In [ ]:
biglst=[]
for i in range(7,19):
    smalllst=[]
    for j in range(0,500):
        str=preprodf.iloc[:,i][j]
        ans=preprocess(str,errlst[j])
        smalllst.append(ans)
    biglst.append(smalllst)

In [ ]:
aaa=pd.DataFrame(biglst).transpose()
aaa.to_csv("collectednow.csv",encoding='euc-kr')
